# Importing Libraries

Discrepancies: The paper hasn't used all of the foods mentioned in the FNDDS Dataset, it has only taken 4889 foods


In [1]:
# 4889 foods compare and create new csv
# Log average, log std, linear avg, linear std for each nutrient which is in Figure1a across 4889 foods

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

In [14]:
raw_apple = '63101000'     #from file MainFoodDesc.csv  #in mg
maraschino_cherry = '63111010'     #from file MainFoodDesc.csv  #in mg
margarine = '81102000'     #from file MainFoodDesc.csv  #in mg
yeast = '75236000'         #from file MainFoodDesc.csv  #in mg
raw_oyester = '26315100'   #from file MainFoodDesc.csv  #in mg


thiamin = '404'         #from file NutDesc.csv
zinc = '309'            #from file NutDesc.csv
protein = '203'         #from file NutDesc.csv
gadoleic_acid = '617'   #from file NutDesc.csv

# Function for converting milligrams, micrograms to grams

In [15]:
def convert_to_grams(row):
    if row['Unit'] == 'mg':
        return row['Amount_x'] / 1000
    elif row['Unit'] == 'mcg':
        return row['Amount_x'] / 1000000
    elif row['Unit'] == 'g':
        return row['Amount_x']
    else:
        return 0

In [30]:
def find_avg(data,food,nutrient):
    num = 0
    denom=0
    for i in data:
        j = i[0].split(',')
        if(j[0]==food and j[1]==nutrient):
            num += float(j[4])
            denom += 1

    return str(num/denom)

In [ ]:
figure1_a = pd.read_csv('CSVs/Figure1a_FinalDF.csv')
figure1_a

# Sorting the dataframe by the column = Amount

In [ ]:
figure1_a['Amount_x'] = figure1_a.apply(convert_to_grams, axis = 1)
figure1_a = figure1_a[figure1_a['Amount_x'] != 0].sort_values(by='Amount_x', ascending=False)
figure1_a 

# Plotting Figure 1a

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(figure1_a['Amount_x'], figure1_a['Nutrient'], marker='o')
x_ticks = [10**i for i in range(-6, 3)]
plt.xscale('log')
plt.xticks(x_ticks, [f"{val:.0e}" for val in x_ticks])
plt.show()

In [18]:
f = open('CSVs/FNDDS.csv', 'r')
# f = open('CSVs/FNDDS.csv', 'r')
data = []

for line in f:
    data.append(line.strip().split(' '))

f.close()

# print("fasdfsdf")

In [19]:
data = data[1:]
data

[['11000000,203,01/01/2009,12/31/2010,1.03,Protein,PROCNT,g,2,"~Milk,',
  'human~"'],
 ['11000000,204,01/01/2009,12/31/2010,4.38,Total',
  'Fat,FAT,g,2,"~Milk,',
  'human~"'],
 ['11000000,205,01/01/2009,12/31/2010,6.89,Carbohydrate,CHOCDF,g,2,"~Milk,',
  'human~"'],
 ['11000000,208,01/01/2009,12/31/2010,70,Energy,ENERC_KCAL,kcal,0,"~Milk,',
  'human~"'],
 ['11000000,221,01/01/2009,12/31/2010,0.0,Alcohol,ALC,g,1,"~Milk,', 'human~"'],
 ['11000000,255,01/01/2009,12/31/2010,87.50,Water,WATER,g,2,"~Milk,',
  'human~"'],
 ['11000000,262,01/01/2009,12/31/2010,0,Caffeine,CAFFN,mg,0,"~Milk,',
  'human~"'],
 ['11000000,263,01/01/2009,12/31/2010,0,Theobromine,THEBRN,mg,0,"~Milk,',
  'human~"'],
 ['11000000,269,01/01/2009,12/31/2010,6.89,"Sugars,',
  'total",SUGAR,g,2,"~Milk,',
  'human~"'],
 ['11000000,291,01/01/2009,12/31/2010,0.0,"Fiber,',
  'total',
  'dietary",FIBTG,g,1,"~Milk,',
  'human~"'],
 ['11000000,301,01/01/2009,12/31/2010,32,Calcium,CA,mg,0,"~Milk,', 'human~"'],
 ['11000000,303,01/01

In [31]:
print("raw_apple: thiamin: "+find_avg(data,raw_apple,thiamin)+"x10^-3")
print("maraschino_cherry: thiamin: "+find_avg(data,maraschino_cherry,thiamin)+"x10^-3")
print("margarine: thiamin: "+find_avg(data,margarine,thiamin)+"x10^-3")
print("raw_oyester: thiamin: "+find_avg(data,raw_oyester,thiamin)+"x10^-3")
print("yeast: thiamin: "+find_avg(data,yeast,thiamin)+"x10^-3")

print()

print("raw_apple: zinc: "+find_avg(data,raw_apple,zinc)+"x10^-3")
print("maraschino_cherry: zinc: "+find_avg(data,maraschino_cherry,zinc)+"x10^-3")
print("margarine: zinc: "+find_avg(data,margarine,zinc)+"x10^-3")
print("raw_oyester: zinc: "+find_avg(data,raw_oyester,zinc)+"x10^-3")
print("yeast: zinc: "+find_avg(data,yeast,zinc)+"x10^-3")

raw_apple: thiamin: 0.017x10^-3
maraschino_cherry: thiamin: 0.0x10^-3
margarine: thiamin: 0.007x10^-3
raw_oyester: thiamin: 0.018x10^-3
yeast: thiamin: 10.99x10^-3

raw_apple: zinc: 0.04x10^-3
maraschino_cherry: zinc: 0.26x10^-3
margarine: zinc: 0.0x10^-3
raw_oyester: zinc: 39.3x10^-3
yeast: zinc: 7.94x10^-3


In [49]:
figure1_c = pd.read_csv('CSVs/Particular Nutrients/ZincThiamin.csv')
figure1_c['Amount'] = figure1_c['Amount'].apply(lambda x: x / 1000) #mg to grams conversion
# figure1_c = figure1_c[figure1_c['Amount'] != 0]

figure1_c = figure1_c.pivot_table(index=['Food Index', 'Food'], columns='Nutrient Index', values='Amount', aggfunc='sum').reset_index()
figure1_c.columns = ['Food Index', 'Food', 'Zinc', 'Thiamin']

figure1_c['Zinc'] = figure1_c['Zinc'].replace(np.nan,0)
figure1_c['Thiamin'] = figure1_c['Thiamin'].replace(np.nan,0)

# figure1_c.to_csv("CSVs/Particular Nutrients/PivotedZincThiamin.csv")

figure1_c

,Food Index,Food,Zinc,Thiamin
0,11000000,"Milk, human",0.00017,0.000014
1,11100000,"Milk, NFS",0.00043,0.000040
2,11111000,"Milk, cow's, fluid, whole",0.00037,0.000046
3,11111100,"Milk, cow's, fluid, whole, low-sodium",0.00038,0.000020
4,11111150,"Milk, calcium fortified, cow's, fluid, whole",0.00037,0.000046
...,...,...,...,...
7248,94100200,"Water, bottled, sweetened, with low or no calo...",0.00000,0.000000
7249,94100300,"Water, fruit flavored, sweetened, with high fr...",0.00032,0.000000
7250,94210100,Propel Water,0.00000,0.000000
7251,94210200,Glaceau Water,0.00032,0.000000


In [13]:
zinc_list = figure1_c['Zinc'].to_list()

def sn_calculator():
    pass

In [14]:
def mn_calculator(linear_average_conc,sn =1.53):
    mn = np.log(linear_average_conc) - (math.pow(sn,2))/2
    return mn

In [15]:
def qn_calculator(linear_conc, sn=1.53):
    mn = mn_calculator(0.000199695*1e3)
    logarithmic_conc = np.log(linear_conc)
    denominator = linear_conc * sn *np.sqrt(2*np.pi)
    numerator = 1/ (np.exp(((logarithmic_conc-mn)**2)/(2*(sn)**2)))
    
    
    return numerator/denominator

In [16]:
food_indices = [63101000, 63111010, 81102000, 75236000, 26315100]

filtered_figure1_c = pd.read_csv("CSVs/Particular Nutrients/PivotedZincThiamin.csv")
filtered_figure1_c = filtered_figure1_c[filtered_figure1_c['Food Index'].isin(food_indices)]

filtered_figure1_c

# Zinc = 0.009508 #Thiamin = 0.0022064
# Zinc = 0.001319465 # Thiamin = 1.85E-04

,Unnamed: 0,Food Index,Food,Zinc,Thiamin
1424,1424,26315100,"Oysters, raw",0.03930,0.000018
4885,4885,63101000,"Apple, raw",0.00004,0.000017
4920,4920,63111010,"Cherries, maraschino",0.00026,0.000000
6211,6211,75236000,Yeast,0.00794,0.010990
6592,6592,81102000,"Margarine, NFS",0.00000,0.000007


In [17]:
qn_calculator(1e-4)

0.38216581501399577

In [41]:
def standard_deviation(data,nutrient):
    log_avg = 0
    log_avg_sq = 0
    c=0
    for i in data:
        j = i[0].split(',')
        if(j[1]==nutrient):
            if(float(j[4]) == 0):
                c+=1
                continue
            log_avg += np.log(float(j[4]))
            log_avg_sq += np.log(float(j[4]))**2
            c+=1

    return ((log_avg_sq/c) - ((log_avg/c)**2))**0.5

In [42]:

print(standard_deviation(data,thiamin))   #1.26543225 is the standard deviation acc to 4889 foods
print(standard_deviation(data,zinc))   #1.34545079 is the standard deviation acc to 4889 foods
print(standard_deviation(data,protein))   #1.34357727 is the standard deviation acc to 4889 foods
print(standard_deviation(data,gadoleic_acid))   #2.1041791 is the standard deviation acc to 4889 foods


# data

1.2960681729618984
1.2987449041275738
1.3200769941915813
2.009582874180106


In [43]:
l = np.array([standard_deviation(data,thiamin),standard_deviation(data,zinc),standard_deviation(data,protein),standard_deviation(data,gadoleic_acid)])
# avg = (standard_deviation(data,thiamin)+standard_deviation(data,zinc)+standard_deviation(data,protein)+standard_deviation(data,gadoleic_acid))/4
avg = np.mean(l)
sd = np.std(l)
print(sd)
print(avg)
print(l)

print("standard deviation of 4 nutrients: "+str(avg)+"+-"+str(sd))  #acc to data it is 1.66+-0.39
# avg

0.30525100875291283
1.48111823636529
[1.29606817 1.2987449  1.32007699 2.00958287]
standard deviation of 4 nutrients: 1.48111823636529+-0.30525100875291283


In [44]:
l=np.array([1.26543225,1.34545079,1.34357727,2.1041791])
avg = np.mean(l)
sd = np.std(l)
print(sd)
print(avg)

print("sd of 4 nutrients: "+str(avg)+"+-"+str(sd))  #acc to data it is 1.66+-0.39           


0.3418875172801219
1.5146598525
sd of 4 nutrients: 1.5146598525+-0.3418875172801219


In [47]:
#skewness
def skewness(data, nutrient):
    log_values = []
    n = 0
    for item in data:
        parts = item[0].split(',')
        if parts[1] == nutrient:
            value = float(parts[4])
            if value != 0:
                log_values.append(np.log(value))
                n += 1
    if n < 3:
        return None  # Skewness is undefined for less than 3 data points

    mean = np.mean(log_values)
    std_dev = np.std(log_values, ddof=1)  # Use ddof=1 for sample standard deviation
    skewness = (n / ((n - 1) * (n - 2))) * np.sum(((log_values - mean) / std_dev) ** 3)
    return skewness


In [48]:
print(skewness(data,thiamin))       #-0.035609511 is the skewness acc to 4889 foods
print(skewness(data,zinc))          #-0.445958592 is the skewness acc to 4889 foods
print(skewness(data,protein))       #-1.108838677 is the skewness acc to 4889 foods
print(skewness(data,gadoleic_acid)) #-1.094805201 is the skewness acc to 4889 foods


# A negative skewness value indicates that the distribution is skewed to the left, 
# with the tail on the left side of the distribution being longer or fatter compared to the right side.


-0.06719423364617477
-0.4654347450506087
-1.0459089614111048
-1.0803797854093047
